In [46]:
# Load some Daysim data, process summaries and push to a Tableau db

In [1]:
from datetime import datetime

In [2]:
datetime.now().strftime("%m/%d/%Y %I:%M:%S %p")

'10/29/2015 09:42:15 AM'

In [5]:
import sqlite3 as lite
import sys
import h5py
import pandas as pd
# from EmmeProject import *

In [6]:
df = pd.DataFrame()

In [10]:
df.append({'a':1},ignore_index=True)

,a
0,1


In [2]:
daysim_base = r'D:\soundcast\soundcast\outputs\daysim_outputs.h5' # 2010 
h5_base = h5py.File(daysim_base)

In [3]:
h5_base

<HDF5 file "daysim_outputs.h5" (mode r+)>

In [4]:
#data_tables = ["Household", "HouseholdDay", "Person", "PersonDay", "Tour", "Trip"]
data_tables = ["Trip"]

In [139]:
trip_cols = [u'dadtyp', u'deptm', u'dorp', u'dpcl', u'dpurp', u'dtaz', u'hhno', u'mode', u'oadtyp', u'opcl', u'opurp', u'otaz', u'pathtype', u'pno', u'travcost', u'travdist', u'travtime', u'vot']

In [140]:
def process_h5(data_table, h5_file, columns):
    ''' Convert DaySim tables (e.g., person, household files) to dataframe ''' 
    df = pd.DataFrame()     # initialize empty data frame
    
    for col in columns:
        df[col] = h5_file[data_table][col].value
        print 'Processing data column: ' + col
    return df

In [348]:
base_df.groupby('mode').mean()[[metric]].T

mode,1,2,3,4,5,6,8
travtime,22.525333,15.301115,16.824169,13.838758,13.450481,38.394325,12.003207


In [141]:
base_df = process_h5('Trip', h5_base, trip_cols)

Processing data column: dadtyp
Processing data column: deptm
Processing data column: dorp
Processing data column: dpcl
Processing data column: dpurp
Processing data column: dtaz
Processing data column: hhno
Processing data column: mode
Processing data column: oadtyp
Processing data column: opcl
Processing data column: opurp
Processing data column: otaz
Processing data column: pathtype
Processing data column: pno
Processing data column: travcost
Processing data column: travdist
Processing data column: travtime
Processing data column: vot


In [350]:
# Let's make some simple summaries for now

In [149]:
(base_df.groupby('mode').count()['travcost']/len(base_df)).sum()

1.0

In [351]:
def testfunc():
    return 5


In [352]:
a = [testfunc()]
a

[5]

In [353]:
for metric in ['travtime']:
    for agg in ['mean', 'max']:
        df = eval("base_df.groupby('mode')." + agg + "()[[metric]].T")
        df.columns = [col + '_' + agg for col in mode_columns]

In [354]:
df

,Walk_max,Bike_max,SOV_max,HOV2_max,HOV3+_max,Transit_max,School Bus_max
travtime,200,131.07,194.01,161.05,167.95,221.24,122.66


In [355]:
mode_columns = ['Walk','Bike','SOV','HOV2','HOV3+','Transit','School Bus']


In [356]:
mode_columns

['Walk', 'Bike', 'SOV', 'HOV2', 'HOV3+', 'Transit', 'School Bus']

In [357]:
[col + '_test' for col in mode_columns]

['Walk_test',
 'Bike_test',
 'SOV_test',
 'HOV2_test',
 'HOV3+_test',
 'Transit_test',
 'School Bus_test']

In [358]:
a = base_df.groupby('mode').mean()[['travcost']]
# add in some labels

# Transpose to get in row format
a = a.T
a.columns = ['Walk','Bike','SOV','HOV2','HOV3+','Transit','School Bus']

In [359]:
a

,Walk,Bike,SOV,HOV2,HOV3+,Transit,School Bus
travcost,0,0,0.719349,0.522752,0.506885,0.964841,0.405587


In [360]:
# Add standard set of columns to this df for filtering
a['model'] ,a['runid'], a['date'], a['model_year'] = ['Soundcast',1,'10/15/2015','2040']

In [361]:
################ What else do we want to summarize
a

,Walk,Bike,SOV,HOV2,HOV3+,Transit,School Bus,model,runid,date,model_year
travcost,0,0,0.719349,0.522752,0.506885,0.964841,0.405587,Soundcast,1,10/15/2015,2040


In [362]:
################ What else do we want to summarize
# Mode share
b = base_df.groupby('mode').count()[['dtaz']]/len(base_df)

# Transpose to get in row format
b = b.T
b.columns = ['Walk','Bike','SOV','HOV2','HOV3+','Transit','School Bus']

# Add standard set of columns to this df for filtering
b['model'] ,b['runid'], b['date'], b['model_year'] = ['Soundcast',2,'10/25/2015','2030']

In [363]:
pd.concat([a,b], axis=1)

,Walk,Bike,SOV,HOV2,HOV3+,Transit,School Bus,model,runid,date,...,Bike,SOV,HOV2,HOV3+,Transit,School Bus,model,runid,date,model_year
dtaz,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.006718,0.479188,0.206022,0.173168,0.034785,0.021405,Soundcast,2,10/25/2015,2030
travcost,0,0,0.719349,0.522752,0.506885,0.964841,0.405587,Soundcast,1,10/15/2015,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [389]:
testdf = pd.DataFrame({'test': [1,2,3,]}).T
testdf

,0,1,2
test,1,2,3


In [396]:
def stamp(mydf):
    mydf['newcol'] = 4
    return mydf

In [398]:
stamp(testdf)

,0,1,2,newcol
test,1,2,3,4


In [364]:
# Trip distance distribution

# average trip distance by mode
c = base_df.groupby('mode').mean()[['travcost']]
# add in some labels

# Transpose to get in row format
c = c.T
c.columns = ['Walk','Bike','SOV','HOV2','HOV3+','Transit','School Bus']

# Add standard set of columns to this df for filtering
c['model'] ,c['runid'], c['date'], c['model_year'] = ['Soundcast',2,'10/25/2015','2030']

In [365]:
# Trip distance distribution
pd.cut(base_df['travdist'], 10)

0           (15.044, 30.088]
1           (15.044, 30.088]
2            (-0.15, 15.044]
3            (-0.15, 15.044]
4            (-0.15, 15.044]
5            (-0.15, 15.044]
6           (15.044, 30.088]
7           (15.044, 30.088]
8            (-0.15, 15.044]
9            (-0.15, 15.044]
10           (-0.15, 15.044]
11           (-0.15, 15.044]
12           (-0.15, 15.044]
13           (-0.15, 15.044]
14           (-0.15, 15.044]
15           (-0.15, 15.044]
16           (-0.15, 15.044]
17           (-0.15, 15.044]
18           (-0.15, 15.044]
19           (-0.15, 15.044]
20           (-0.15, 15.044]
21           (-0.15, 15.044]
22           (-0.15, 15.044]
23           (-0.15, 15.044]
24           (-0.15, 15.044]
25           (-0.15, 15.044]
26           (-0.15, 15.044]
27           (-0.15, 15.044]
28           (-0.15, 15.044]
29          (15.044, 30.088]
                  ...       
21455477     (-0.15, 15.044]
21455478     (-0.15, 15.044]
21455479     (-0.15, 15.044]
21455480     (

In [366]:
# average trip cost

In [367]:
# average trip length

In [368]:
# person and household level stuff?

In [369]:
# VMT

In [370]:
# VHT

In [371]:
# Delay

In [372]:
# Speed

In [373]:
# Tolls

In [374]:
# Travel Times

In [375]:
# Transit Routes

In [376]:
# Screenlines

In [377]:
# Trip purposes

In [378]:
# # Export this as a row to a db
# con = None
# con = lite.connect('mynewdb.sl3')
# for existing dbs, change if_exists replace to append
# a.to_sql(name='mynewdb.sl3', con=con, if_exists='replace', chunksize=1000)

In [379]:
# What really needs to be happening is that this data gets written out a SQL db
# that is synched up with Tableau

# What we could do for now is just add a row of the new data to a Tableau extract?

In [380]:
a

,Walk,Bike,SOV,HOV2,HOV3+,Transit,School Bus,model,runid,date,model_year
travcost,0,0,0.719349,0.522752,0.506885,0.964841,0.405587,Soundcast,1,10/15/2015,2040


In [381]:
b

,Walk,Bike,SOV,HOV2,HOV3+,Transit,School Bus,model,runid,date,model_year
dtaz,0.078713,0.006718,0.479188,0.206022,0.173168,0.034785,0.021405,Soundcast,2,10/25/2015,2030


In [382]:
# Make up some fake data for a second row
b = base_df.groupby('mode').mean()[['travcost']]-.04
# add in some labels

# Transpose to get in row format
b = b.T
b.columns = ['Walk','Bike','SOV','HOV2','HOV3+','Transit','School Bus']

In [383]:
# Add standard set of columns to this df for filtering
b['model'] ,b['runid'], b['date'], b['model_year'] = ['Soundcast',2,'10/25/2015','2030']

In [384]:
# # Export this as a row to a db
con = None
con = lite.connect('mynewdb.sl3')
b.to_sql(name='mynewdb.sl3', con=con, if_exists='append', chunksize=1000)

c:\Users\Brice\AppData\Local\Continuum\Anaconda\lib\site-packages\pandas\io\sql.py:1371: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  warnings.warn(_SAFE_NAMES_WARNING)


In [385]:
####################### Get network attributes and export results

In [399]:
my_project = EmmeProject(r'D:\soundcast\soundcast\projects\LoadTripTables\LoadTripTables.emp')

RuntimeError: Modeller instance already created.

In [ ]:
con = None
con = lite.connect('mynewdb.sl3')

In [ ]:
from datetime import datetime

In [ ]:
datetime.now()

In [ ]:
log = pd.read_csv(r'D:\soundcast\soundcast\soundcast_log.txt')

In [ ]:
timestamp = str(pd.read_csv(r'D:\soundcast\soundcast\soundcast_log.txt').iloc[-1]).split(' ')

In [ ]:
timestamp[1] + ' ' + timestamp[2]

In [ ]:
def testfunc():
    return 4, 5

In [ ]:
testfunc()[1]

In [ ]:
testfunc()

In [400]:
datetime.now().strftime("%m/%d/%Y %I:%M:%S %p")

'10/27/2015 09:40:13 AM'

In [2]:
from EmmeProject import *

In [4]:
my_project = EmmeProject(r'D:\Maps\soundcast\projects\LoadTripTables\LoadTripTables.emp')

In [7]:
my_project.change_active_database('7to8')

In [8]:
my_project

<EmmeProject.EmmeProject instance at 0x000000000E3594C8>

In [9]:
network = my_project.current_scenario.get_network()

In [10]:
my_project.network_calculator("link_calculation",result = None, expression = "@vmt")

In [11]:
mydict = {}

In [12]:
mydict['7to8'] = 4567
mydict

{'7to8': 4567}

In [13]:
my_project.network_calc_result['sum']

0.0

In [14]:
tods = [bank.title() for bank in my_project.data_explorer.databases()]

In [15]:
fac_type_dict = {'highway' : 'ul3 = 1 or ul3 = 2',
                 'arterial' : 'ul3 = 3 or ul3 = 4 or ul3 = 6',
                 'connectors' : 'ul3 = 5'}

In [16]:
test = {}

In [17]:
test['a'] = 5

In [18]:
tod_dict = {}
for tod in ['7to8']:
    my_project.change_active_database(tod)
    network = my_project.current_scenario.get_network()

    fac_dict = {}
    for expr in ['@vmt']:
        for key, value in fac_type_dict.iteritems():
            results = {}
            my_project.network_calculator(
                "link_calculation", 
                result=None, 
                expression=expr,  
                selections_by_link=value)
            results[key] = my_project.network_calc_result['sum']

            

    tod_dict[tod] = results
test = pd.DataFrame(tod_dict)

In [19]:
tod_dict = {}
for tod in tods:
    my_project.change_active_database(tod)
    network = my_project.current_scenario.get_network()

    fac_dict = {}

    for key, value in fac_type_dict.iteritems():
        fac_dict[key] = {}
        for expr in ['@vmt','@vht']:
            my_project.network_calculator(
                "link_calculation", 
                result=None, 
                expression=expr,  
                selections_by_link=value)
            fac_dict[key][expr] = my_project.network_calc_result['sum']            
    tod_dict[tod] = fac_dict

In [18]:
df =pd.DataFrame(tod_dict['7to8'])

In [19]:
df.ix['@vht']

arterial      75992.000000
connectors    18303.074219
highway       56083.828125
Name: @vht, dtype: float64

In [20]:
pd.DataFrame(tod_dict['7to8']).ix['@vht'].sum()

150378.902344

In [21]:
pd.DataFrame(tod_dict['7to8']).ix['@vht']

arterial      75992.000000
connectors    18303.074219
highway       56083.828125
Name: @vht, dtype: float64

In [22]:
hourly_vmt = {}

In [23]:
hourly_vmt['7to8'] = pd.DataFrame(tod_dict['7to8']).ix['@vmt'].sum()

In [24]:
hourly_vmt = {}
for tod in tods:
    # Total for all facilities by time of day
    hourly_vmt[tod] = pd.DataFrame(tod_dict[tod]).ix['@vmt'].sum()

In [25]:
pd.DataFrame(hourly_vmt, index=[0])

,10to14,14to15,15to16,16to17,17to18,18to20,20to5,5to6,6to7,7to8,8to9,9to10
0,18698194.125,5240210.625,5945089.28125,5519015.25,6108506.46875,8884616.0625,11376978.9375,1835767.304688,3406715.015625,4656979.09375,3962214.046875,4125965.8125


In [20]:
tod_dict['7to8']['highway']['@vht']

0.0

In [21]:
vmt_vht=tod_dict

In [22]:
vmt_vht['7to8']['arterial']['@vmt']

0.0

In [23]:
# vmt_vht['7to8']['arterial']['@vmt']
vmt_vht['18to20']['arterial']

{'@vht': 161692.328125, '@vmt': 4256450.5}

In [24]:
fac_vmt = {}
fac_vht = {}



for tod in tods:
    fac_dict_vmt = {}
    fac_dict_vht = {}

    # Total for all times of day by facility type
    for fac_type in ['highway', 'arterial', 'connectors']:
        fac_dict_vmt[fac_type] = vmt_vht[tod][fac_type]['@vmt']
        fac_dict_vht[fac_type] = vmt_vht[tod][fac_type]['@vht']
        
    fac_vmt[tod] = fac_dict_vmt
    fac_vht[tod] = fac_dict_vht
    

In [27]:
pd.DataFrame(pd.DataFrame(fac_vht).sum()).T

,10to14,14to15,15to16,16to17,17to18,18to20,20to5,5to6,6to7,7to8,8to9,9to10
0,596460.953125,170217.791016,205975.804688,186342.566407,219607.234376,293365.023437,339514.820312,48691.894532,97809.085937,0,119868.777343,125988.806641


In [33]:
vmt_tod=pd.DataFrame(pd.DataFrame(fac_vht).T.sum()).T
# vmt_tod.T
# b=a.T

In [117]:
a

,10to14,14to15,15to16,16to17,17to18,18to20,20to5,5to6,6to7,7to8,8to9,9to10
arterial,8567331.000,2398790.000,2824212.00000,2540470.00,2909993.75000,4256450.5000,4899728.0000,677038.500000,1330565.375000,2043806.00000,1674342.625000,1763401.3750
connectors,1443831.125,385995.375,440324.78125,371387.00,411696.71875,684282.0625,731143.4375,95924.929688,169133.515625,280998.34375,246893.171875,273600.5625
highway,8687032.000,2455425.250,2680552.50000,2607158.25,2786816.00000,3943883.5000,5746107.5000,1062803.875000,1907016.125000,2332174.75000,2040978.250000,2088963.8750


In [126]:
def get_unique_screenlines(project):
    network = project.current_scenario.get_network()
    unique_screenlines = []
    for link in network.links():
        if link.type <> 90 and link.type not in unique_screenlines:
            unique_screenlines.append(str(link.type))
    return unique_screenlines


In [128]:
screenline_list  = get_unique_screenlines(my_project)

In [133]:
extra_attributes_dict = {'@tveh' : 'total vehicles', 
                         '@mveh' : 'medium trucks', 
                         '@hveh' : 'heavy trucks', 
                         '@vmt' : 'vmt',\
                         '@vht' : 'vht', 
                         '@trnv' : 'buses in auto equivalents',
                         '@ovol' : 'observed volume', 
                         '@bveh' : 'number of buses'}

In [134]:
screenline_dict = {}
    
for item in screenline_list:
    #dict where key is screen line id and value is 0
    screenline_dict[item] = 0

#loop through all tod banks and get network summaries
for key in tods:
    my_project.change_active_database(key)
    for name, desc in extra_attributes_dict.iteritems():
        my_project.create_extra_attribute('LINK', name, desc, 'True')
    #TRANSIT:
    if my_project.tod in transit_tod.keys():
        for name, desc in transit_extra_attributes_dict.iteritems():
            my_project.create_extra_attribute('TRANSIT_LINE', name, desc, 'True')
        calc_transit_link_volumes(my_project)
        calc_transit_line_atts(my_project)

        transit_summary_dict[key] = get_transit_boardings_time(my_project)
        #print transit_summary_dict

NameError: name 'transit_tod' is not defined

In [135]:
base_df

NameError: name 'base_df' is not defined

In [ ]:
def corridor_results(tod, hour, my_project):
    corridor_count = 12    # number of input corridor files

    # filepath = r'projects\\' + tod + '\\' + tod + '.emp'
    # my_project = EmmeProject(filepath)
    my_project.change_active_database(hour)

    # Access the nework link data
    network = my_project.current_scenario.get_network()

    # Get the auto time and length of each link
    

    # Get dataframes for time and length
    time_df = get_link_attribute(network=network, attr='auto_time')
    length_df = get_link_attribute(network=network, attr='length')    

    # combine link time and length data into single dataframe
    link_df = pd.merge(time_df, length_df)

    corridor_flags_df = pd.DataFrame()
    for i in range(1, corridor_count+1):    # +1 because python is zero-based
        corridor_df = pd.read_table(r'inputs/corridors/corridor_' + str(i) + '.in', skiprows=1, skipinitialspace=True, sep=' ')
        corridor_df['link_id'] = corridor_df['inode'].astype('str') + '-' + corridor_df['jnode'].astype('str')
        corridor_flags_df = pd.concat(objs=[corridor_flags_df, corridor_df])

    corridor_flags_df.fillna(0, inplace=True)

    # join corridor flags to link travel time
    corridor_times_df = pd.merge(link_df, corridor_flags_df)

    # sum corridor travel time and length for each corridor
    link_trav_time = pd.DataFrame()
    for i in range(1, corridor_count+1):    # +1 because python is zero-based
        if i < 10:
            code = '@corr'
        else:
            code = '@cor'

        corridor_sum = pd.DataFrame(corridor_times_df.groupby(code + str(i)).sum()[['auto_time', 'length']])
        
        # add a corridor id and facility type for analysis in Excel
        corridor_sum['Corridor Input File'] = i
        corridor_sum['Local ID'] = corridor_sum.index
        if i == 1: 
            corridor_sum['corridor_type'] = 'a'    # arterial corridor
        else:
            corridor_sum['corridor_type'] = 'f'    # freeway corridor
        link_trav_time = pd.concat([link_trav_time, corridor_sum])        

    # remove all the 0-index results (these are travel times on non-tagged links)
    link_trav_time = link_trav_time.query('index > 0')

    # Add a column for time of day
    link_trav_time['tod'] = tod

    # Add a column that concatenates the corridor file number and the corridor tag ID 
    # for processessing in Excel
     # (must convert numbers to int then convert to string to join with other strings)
    link_trav_time['full_id'] = link_trav_time['Corridor Input File'].astype('int').astype('str') + link_trav_time['Local ID'].astype('int').astype('str')
    link_trav_time['full_id'] = link_trav_time['corridor_type'] + link_trav_time['tod'] + link_trav_time['full_id']

    

    # Write out to CSV
    df_out = link_trav_time[['tod', 'Corridor Input File', 'Local ID', 
                'full_id', 'auto_time', 'length']]

    return df_out

In [2]:
#Copyright [2014] [Puget Sound Regional Council]

#Licensed under the Apache License, Version 2.0 (the "License");
#you may not use this file except in compliance with the License.
#You may obtain a copy of the License at

#    http://www.apache.org/licenses/LICENSE-2.0

#Unless required by applicable law or agreed to in writing, software
#distributed under the License is distributed on an "AS IS" BASIS,
#WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#See the License for the specific language governing permissions and
#limitations under the License.

import array as _array
import inro.emme.desktop.app as app
import inro.modeller as _m
import inro.emme.matrix as ematrix
import inro.emme.database.matrix
import inro.emme.database.emmebank as _eb
import json
import numpy as np
import time
import os,sys
import h5py
import Tkinter, tkFileDialog
import multiprocessing as mp
import subprocess
import csv
import xlsxwriter
# import xlautofit
import sqlite3 as lite
import datetime
from scripts.EmmeProject import *

from multiprocessing import Pool
import pandas as pd
sys.path.append(os.path.join(os.getcwd(),"inputs"))
sys.path.append(os.path.join(os.getcwd(),"scripts"))
from input_configuration import *

network_summary_project = 'Projects/LoadTripTables/LoadTripTables.emp'
fac_type_dict = {'highway' : 'ul3 = 1 or ul3 = 2',
                 'arterial' : 'ul3 = 3 or ul3 = 4 or ul3 = 6',
                 'connectors' : 'ul3 = 5'}

extra_attributes_dict = {'@tveh' : 'total vehicles', 
                         '@mveh' : 'medium trucks', 
                         '@hveh' : 'heavy trucks', 
                         '@vmt' : 'vmt',\
                         '@vht' : 'vht', 
                         '@trnv' : 'buses in auto equivalents',
                         '@ovol' : 'observed volume', 
                         '@bveh' : 'number of buses'}

transit_extra_attributes_dict = {'@board' : 'total boardings', '@timtr' : 'transit line time'}

transit_tod = {'6to7' : {'4k_tp' : 'am', 'num_of_hours' : 1}, 
               '7to8' :  {'4k_tp' : 'am', 'num_of_hours' : 1}, 
               '8to9' :  {'4k_tp' : 'am', 'num_of_hours' : 1}, 
               '9to10' : {'4k_tp' : 'md', 'num_of_hours' : 1}, 
               '10to14' : {'4k_tp' : 'md', 'num_of_hours' : 4}, 
               '14to15' : {'4k_tp' : 'md', 'num_of_hours' : 1}}
# Input Files:
aadt_counts_file = 'soundcast_aadt.csv'
tptt_counts_file = 'soundcast_tptt.csv'

uc_list = ['@svtl1', '@svtl2', '@svtl3', '@svnt1', '@svnt2', '@svnt3', '@h2tl1', '@h2tl2', '@h2tl3',
           '@h2nt1', '@h2nt2', '@h2nt3', '@h3tl1', '@h3tl2', '@h3tl3', '@h3nt1', '@h3nt2', '@h3nt3', '@lttrk', '@mveh', '@hveh', '@bveh']

def get_link_attribute(network, attr):
    ''' Return dataframe of link attribute and link ID'''
    link_dict = {}
    for i in network.links():
        link_dict[i.id] = i[attr]
    df = pd.DataFrame({'link_id': link_dict.keys(), attr: link_dict.values()})
    return df

def json_to_dictionary(dict_name):

    #Determine the Path to the input files and load them
    skim_params_loc = os.path.abspath(os.path.join(os.getcwd(),"inputs\\skim_params"))    # Assumes the cwd is @ run_soundcast.py; always run this script from run_soundcast.py
    input_filename = os.path.join(skim_params_loc,dict_name+'.json').replace("\\","/")
    my_dictionary = json.load(open(input_filename))

    return(my_dictionary)
 
def calc_vmt_vht_delay_by_ft(EmmeProject):
    ###calculates vmt, vht, and delay for all links and returns a nested dictionary with key=metric(e.g. 'vmt') 
    #and value = dictionary where dictionary has key = facility type(e.g. 'highway') and value = sum of metric 
    #for that facility type
  
     #medium trucks
     EmmeProject.network_calculator("link_calculation", result = '@mveh', expression = '@metrk/1.5')
     
     #heavy trucks:
     EmmeProject.network_calculator("link_calculation", result = '@hveh', expression = '@hvtrk/2')
     
     #busses:
     EmmeProject.network_calculator("link_calculation", result = '@bveh', expression = '@trnv/2')
     ####################still need to do*****************************
     #hdw- number of buses:
     #mod_spec = network_calc_spec
     #mod_spec["result"] = "@hdw"
     #mod_spec["expression"] = 'hdw'
     #network_calc(mod_spec)
     
     #calc total vehicles, store in @tveh 
     str_expression = '@svtl1 + @svtl2 + @svtl3 + @svnt1 +  @svnt2 + @svnt3 + @h2tl1 + @h2tl2 + @h2tl3 + @h2nt1 + @h2nt2 + @h2nt3 + @h3tl1\
                       + @h3tl2 + @h3tl3 + @h3nt1 + @h3nt2 + @h3nt3 + @lttrk + @mveh + @hveh + @bveh'
     EmmeProject.network_calculator("link_calculation", result = '@tveh', expression = str_expression)
     #a dictionary to hold vmt/vht/delay values:
     results_dict = {}
     #dictionary to hold vmts:
     vmt_dict = {}
     #calc vmt for all links by factilty type and get sum by ft. 
     for key, value in fac_type_dict.iteritems():    
        EmmeProject.network_calculator("link_calculation", result = "@vmt", expression = "@tveh * length", selections_by_link = value)
        #total vmt by ft: 
        vmt_dict[key] = EmmeProject.network_calc_result['sum']
     #add to results dictionary
     results_dict['vmt'] = vmt_dict
    
     #Now do the same for VHT:
     vht_dict = {}
     for key, value in fac_type_dict.iteritems():    
        EmmeProject.network_calculator("link_calculation", result = "@vht", expression = "@tveh * timau / 60", selections_by_link = value)
        vht_dict[key] = EmmeProject.network_calc_result['sum']
     results_dict['vht'] = vht_dict

     #Delay:
     delay_dict = {}
     for key, value in fac_type_dict.iteritems():    
        EmmeProject.network_calculator("link_calculation",result = None, expression =  "@tveh*(timau-(length*60/ul2))/60", selections_by_link = value)
        delay_dict[key] = EmmeProject.network_calc_result['sum']
     
     results_dict['delay'] = delay_dict
     return results_dict

def vmt_by_user_class(EmmeProject):
    #uc_list = ['@svtl1', '@svtl2', '@svtl3', '@svnt1', '@h2tl1', '@h2tl2', '@h2tl3', '@h2nt1', '@h3tl1', '@h3tl2', '@h3tl3', '@h3nt1', '@lttrk', '@mveh', '@hveh', '@bveh']
    uc_vmt_list = []
    for item in uc_list:
        EmmeProject.network_calculator("link_calculation", result = None, expression = item + ' * length')
        #total vmt by ft: 
        uc_vmt_list.append(EmmeProject.network_calc_result['sum'])
    return uc_vmt_list

def get_link_counts(EmmeProject, df_counts, tod):
    #get the network for the active scenario
     network = EmmeProject.current_scenario.get_network()
     list_model_vols = []
     for item in df_counts.index:
         i = list(item)[0]
         j = list(item)[1]
         link = network.link(i, j)
         x = {}
         x['loop_INode'] = i
         x['loop_JNode'] = j
         if link <> None:
            x['vol' + tod] = link['@tveh']   
         else:
            x['vol' + tod] = None
         list_model_vols.append(x)
     print len(list_model_vols)
     df =  pd.DataFrame(list_model_vols)
     df = df.set_index(['loop_INode', 'loop_JNode'])
     return df

def get_aadt_volumes(EmmeProject, df_aadt_counts, vol_dict):
    network = EmmeProject.current_scenario.get_network()
    for index, row in df_aadt_counts.iterrows():
        x = {}
        id = row['MIN_ID']
        i = row['MIN_NewINode']
        j = row['MIN_NewJNode']
        if row['MIN_Oneway'] == 2:
            link1 = network.link(i,j)
            link2 = network.link(j, i)
            if link1<>None and link2<> None:
                vol = link1['@tveh'] + link2['@tveh']
            elif link1 == None and link2 == None:
                vol = 0
                #print i, j
            elif link1 <> None and link2 == None:
                vol = link1['@tveh'] 
                #print j, i
            elif link1 == None and link2 <> None:
                vol = link2['@tveh'] 

        elif row['MIN_Oneway'] == 0:
            link1 = network.link(i,j)
            if link1 <> None:
                vol = link1['@tveh']
        else:
            link1 = network.link(j,i)
            if link1 <> None:
                vol = link1['@tveh']

        #hov
        if row['MIN_HOV_I'] > 0:
            i = row['MIN_HOV_I'] + 4000
            j = row['MIN_HOV_J'] + 4000
            #both directions:
            if row['MIN_Oneway'] == 2:
                link1 = network.link(i,j)
                link2 = network.link(j, i)
                if link1<>None and link2<> None:
                    vol = vol +link1['@tveh'] + link2['@tveh']
                elif link1 == None and link2 == None:
                    vol = vol + 0
                    #print i, j
                elif link1 <> None and link2 == None:
                    vol = vol + link1['@tveh'] 
                    #print j, i
                elif link1 == None and link2 <> None:
                    vol = vol + link2['@tveh'] 
            #IJ
            elif row['MIN_Oneway'] == 0:
                link1 = network.link(i,j)
                if link1 <> None:
                    vol = vol + link1['@tveh']
            #JI
            else:
                link1 = network.link(j,i)
                if link1 <> None:
                    vol = vol + link1['@tveh']


        if id in vol_dict.keys():
            vol_dict[id]['EstVol'] = vol_dict[id]['EstVol'] + vol
        else:
            x['ID'] = id
            x['PSRCEdgeID'] = row['PSRCEdgeID']
            x['ObsVol'] = row['MEAN_AADT']
            #x['RteID'] = row['First_Route_ID']
            x['EstVol'] = vol
            vol_dict[id] = x
    return vol_dict

def get_tptt_volumes(EmmeProject, df_tptt_counts, vol_dict):
    network = EmmeProject.current_scenario.get_network()
    for index, row in df_tptt_counts.iterrows():
        x = {}
        id = row ['ID']
        i = row['NewINode']
        j = row['NewJNode']
        if row['Direction_'] == 'Bothways':
            link1 = network.link(i,j)
            link2 = network.link(j, i)
            if link1<>None and link2<> None:
                vol = link1['@tveh'] + link2['@tveh']
            elif link1 == None and link2 == None:
                vol = 0
                #print i, j
            elif link1 <> None and link2 == None:
                vol = link1['@tveh'] 
                #print j, i
            elif link1 == None and link2 <> None:
                vol = link2['@tveh'] 

        elif row['Oneway'] == 0:
            link1 = network.link(i,j)
            if link1 <> None:
                vol = link1['@tveh']
        else:
            link1 = network.link(j,i)
            if link1 <> None:
                vol = link1['@tveh']

        if id in vol_dict.keys():
            vol_dict[id]['EstVol'] = vol_dict[id]['EstVol'] + vol
        else:
            x['ID'] = id
            x['SRID'] = row['SRID']
            x['ObsVol'] = row['Year_2010']
            x['Location'] = row['Location']
            x['EstVol'] = vol
            vol_dict[id] = x
    return vol_dict

def get_unique_screenlines(EmmeProject):
    network = EmmeProject.current_scenario.get_network()
    unique_screenlines = []
    for link in network.links():
        if link.type <> 90 and link.type not in unique_screenlines:
            unique_screenlines.append(str(link.type))
    return unique_screenlines

def get_screenline_volumes(screenline_dict, EmmeProject):

    for screen_line in screenline_dict.iterkeys():
        EmmeProject.network_calculator("link_calculation",result = None, expression = "@tveh", selections_by_link = screen_line)
        screenline_dict[screen_line] = screenline_dict[screen_line] + EmmeProject.network_calc_result['sum']

def calc_transit_line_atts(EmmeProject):
    #calc boardings and transit line time
     EmmeProject.transit_line_calculator(result = '@board', expression = 'board')
     EmmeProject.transit_line_calculator(result = '@timtr', expression = 'timtr')

def get_transit_boardings_time(EmmeProject):
    network = EmmeProject.current_scenario.get_network()
    df_transit_atts = pd.DataFrame(columns=('id', EmmeProject.tod + '_boardings', EmmeProject.tod + '_boardings''_time'))
    line_list = []
    
    for transit_line in network.transit_lines():
        x = {}
        #company_code = transit_line['@ut3']
        x['id'] = transit_line.id
        x[EmmeProject.tod + '_board'] = transit_line['@board']
        x[EmmeProject.tod + '_time']= transit_line['@timtr']
        line_list.append(x)
    df = pd.DataFrame(line_list)
    df = df.set_index(['id'])
    return df

def calc_transit_link_volumes(EmmeProject):
    total_hours = transit_tod[EmmeProject.tod]['num_of_hours']
    my_expression = str(total_hours) + ' * vauteq * (60/hdw)'
    print my_expression
    EmmeProject.transit_segment_calculator(result = '@trnv', expression = my_expression, aggregation = "+")
    
          
        
def writeCSV(fileNamePath, listOfTuples):
    myWriter = csv.writer(open(fileNamePath, 'wb'))
    for l in listOfTuples:
        myWriter.writerow(l)

def dict_to_df(input_dict, measure):
    '''converts triple-nested dict into Dataframe for a given facility type'''
    mydict = {}
    for tod in tods:
        mydict[tod] = {}
        for facility in fac_type_dict.keys():
            mydict[tod][facility] = input_dict[tod][measure][facility]
    return pd.DataFrame(mydict)

def get_runid(table, con):
    '''Update run ID from existing database'''
    try:
        return len(pd.read_sql('select * from ' + table, con))
    except:
        return 0

def get_date():
    '''Get last time stamp from run log.
       Log time stamps are consistently formatted & exist for each line in the log
       For runs without a log, or on error, get current time. '''
    try:
        timestamp = str(pd.read_csv(main_log_file).iloc[-1]).split(' ')
        date = timestamp[0]
        time = timestamp[1] + " " + timestamp[2]
    except:
        date = datetime.now().strftime("%m/%d/%Y")
        time = datetime.now().strftime("%I:%M:%S %p")
    return date, time

def stamp(df, con, table):
    '''Add run information to a table row'''
    df['scenario_name'],df['runid'],df['date'],df['time'] = \
    [scenario_name,get_runid(table, con),get_date()[0],get_date()[1]]
    return df

def export_corridor_results(my_project):
    ''' Evaluate corridor travel time for a single AM and PM period'''
    tod = {'am': '7to8', 'pm': '16to17'}
    am_df = corridor_results(tod=tod['am'], my_project=my_project)
    pm_df = corridor_results(tod=tod['pm'], my_project=my_project)

    # combine am and pm into single CSV and export
    corridor_df = pd.concat(objs=[am_df, pm_df])
    corridor_df.to_csv('outputs/corridor_summary.csv')

def corridor_results(tod, my_project):
    corridor_count = 12    # number of input corridor files

    # filepath = r'projects\\' + tod + '\\' + tod + '.emp'
    # my_project = EmmeProject(filepath)
    my_project.change_active_database(tod)

    # Access the nework link data
    network = my_project.current_scenario.get_network()

    # Get the auto time and length of each link
    

    # Get dataframes for time and length
    time_df = get_link_attribute(network=network, attr='auto_time')
    length_df = get_link_attribute(network=network, attr='length')    

    # combine link time and length data into single dataframe
    link_df = pd.merge(time_df, length_df)

    corridor_flags_df = pd.DataFrame()
    for i in range(1, corridor_count+1):    # +1 because python is zero-based
        corridor_df = pd.read_table(r'inputs/corridors/corridor_' + str(i) + '.in', skiprows=1, skipinitialspace=True, sep=' ')
        corridor_df['link_id'] = corridor_df['inode'].astype('str') + '-' + corridor_df['jnode'].astype('str')
        corridor_flags_df = pd.concat(objs=[corridor_flags_df, corridor_df])

    corridor_flags_df.fillna(0, inplace=True)

    # join corridor flags to link travel time
    corridor_times_df = pd.merge(link_df, corridor_flags_df)

    # sum corridor travel time and length for each corridor
    link_trav_time = pd.DataFrame()
    for i in range(1, corridor_count+1):    # +1 because python is zero-based
        if i < 10:
            code = '@corr'
        else:
            code = '@cor'

        corridor_sum = pd.DataFrame(corridor_times_df.groupby(code + str(i)).sum()[['auto_time', 'length']])
        
        # add a corridor id tag for analysis
        corridor_sum['Corridor Input File'] = i
        corridor_sum['Local ID'] = corridor_sum.index
        link_trav_time = pd.concat([link_trav_time, corridor_sum])        

    # remove all the 0-index results (these are travel times on non-tagged links)
    link_trav_time = link_trav_time.query('index > 0')

    # Add a column that concatenates the corridor file number and the corridor tag ID 
    # for processessing in Excel
    link_trav_time['full_id'] = link_trav_time['Corridor Input File'].astype('str') + link_trav_time['Local ID'].astype('str')
    link_trav_time['full_id'] = link_trav_time['full_id'].astype('float')

    # Add a column for time of day
    link_trav_time['tod'] = tod

    # Write out to CSV
    df_out = link_trav_time[['tod', 'Corridor Input File', 'Local ID', 
                'full_id', 'auto_time', 'length']]

    return df_out


In [3]:
ft_summary_dict = {}
transit_summary_dict = {}

tableau_dict = {}

# Connect to sqlite3 db
con = None
con = lite.connect(results_db)

################################
# UPDATE this path
my_project = EmmeProject(r'd:\Maps\soundcast\projects\LoadTripTables\LoadTripTables.emp')

writer = pd.ExcelWriter('outputs/network_summary_detailed.xlsx', engine = 'xlsxwriter')#Defines the file to write to and to use xlsxwriter to do so
#create extra attributes:


#pandas dataframe to hold count table:
df_counts = pd.read_csv('inputs/network_summary/' + counts_file, index_col=['loop_INode', 'loop_JNode'])
df_aadt_counts = pd.read_csv('inputs/network_summary/' + aadt_counts_file)
df_tptt_counts = pd.read_csv('inputs/network_summary/' + tptt_counts_file)


counts_dict = {}
uc_vmt_dict = {}
aadt_counts_dict = {}

tptt_counts_dict = {}

#get a list of screenlines from the bank/scenario
screenline_list = get_unique_screenlines(my_project) 
screenline_dict = {}

for item in screenline_list:
    #dict where key is screen line id and value is 0
    screenline_dict[item] = 0

#loop through all tod banks and get network summaries
for key, value in sound_cast_net_dict.iteritems():
    my_project.change_active_database(key)
    for name, desc in extra_attributes_dict.iteritems():
        my_project.create_extra_attribute('LINK', name, desc, 'True')
    #TRANSIT:
    if my_project.tod in transit_tod.keys():
        for name, desc in transit_extra_attributes_dict.iteritems():
            my_project.create_extra_attribute('TRANSIT_LINE', name, desc, 'True')
        calc_transit_link_volumes(my_project)
        calc_transit_line_atts(my_project)

        transit_summary_dict[key] = get_transit_boardings_time(my_project)
        #print transit_summary_dict

    net_stats = calc_vmt_vht_delay_by_ft(my_project)

    #store tod network summaries in dictionary where key is tod:
    ft_summary_dict[key] = net_stats
    #store vmt by user class in dict:
    uc_vmt_dict[key] = vmt_by_user_class(my_project)

    #counts:
    df_tod_vol = get_link_counts(my_project, df_counts, key)
    counts_dict[key] = df_tod_vol

    #AADT Counts:

    get_aadt_volumes(my_project, df_aadt_counts, aadt_counts_dict)

    #TPTT:
    get_tptt_volumes(my_project, df_tptt_counts, tptt_counts_dict)


    #screen lines
    get_screenline_volumes(screenline_dict, my_project)


# # Write results to sqlite3 db (for Tableau)
# facility_results = {}
list_of_measures = ['vmt', 'vht', 'delay']
# for measure in list_of_measures:
#     facility_results[measure] = dict_to_df(input_dict=ft_summary_dict, 
#                                             measure=measure)
#     # Mesaures by TOD
#     df = pd.DataFrame(facility_results[measure].sum(),columns=[measure]).T
#     df = stamp(df, con=con, table=measure+'_by_tod')
#     df.to_sql(name=measure+'_by_tod', con=con, if_exists='append', chunksize=1000)

#     # Measures by Facility Type
#     df = pd.DataFrame(facility_results[measure].T.sum(),columns=[measure]).T
#     df = stamp(df, con=con, table=measure+'_by_facility')
#     df.to_sql(name=measure+'_by_facility', con=con, if_exists='append', chunksize=1000)


#write out transit:
print uc_vmt_dict
col = 0
transit_df = pd.DataFrame()

for tod, df in transit_summary_dict.iteritems():
   #if transit_tod[tod] == 'am':
   #    pd.concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False,
   #keys=None, levels=None, names=None, verify_integrity=False)

   workbook = writer.book
   index_format = workbook.add_format({'align': 'left', 'bold': True, 'border': True})
   transit_df = pd.merge(transit_df, df, 'outer', left_index = True, right_index = True)
   #transit_df[tod + '_board'] = df[tod + '_board']
   #transit_df[tod + '_time'] = df[tod + '_time']
transit_df = transit_df[['6to7_board', '6to7_time', '7to8_board', '7to8_time', '8to9_board', '8to9_time', '9to10_board', '9to10_time', '10to14_board', '10to14_time', '14to15_board', '14to15_time']]
transit_df.to_excel(excel_writer = writer, sheet_name = 'Transit Summaries')

   #if col == 0:
   #    worksheet = writer.sheets['Transit Summaries']
   #    routes = df.index.tolist()
   #    for route_no in range(len(routes)):
   #        worksheet.write_string(route_no + 1, 0, routes[route_no], index_format)
   #    col = col + 1
   #    df.to_excel(excel_writer = writer, sheet_name = 'Transit Summaries', index = False, startcol = col)
   #    col = col + 2
   #else:
   #    df.to_excel(excel_writer = writer, sheet_name = 'Transit Summaries', index = False, startcol = col)
   #    col = col + 2


#*******write out counts:
for value in counts_dict.itervalues():
    df_counts = df_counts.merge(value, right_index = True, left_index = True)
    df_counts = df_counts.drop_duplicates()

#write counts out to xlsx:
#loops
df_counts.to_excel(excel_writer = writer, sheet_name = 'Counts Output')

#aadt:
aadt_df = pd.DataFrame.from_dict(aadt_counts_dict, orient="index")
aadt_df.to_excel(excel_writer = writer, sheet_name = 'Arterial Counts Output')

#tptt:
tptt_df = pd.DataFrame.from_dict(tptt_counts_dict, orient="index")
tptt_df.to_excel(excel_writer = writer, sheet_name = 'TPTT Counts Output')



#*******write out network summaries
soundcast_tods = sound_cast_net_dict.keys
list_of_FTs = fac_type_dict.keys()
row_list = []
list_of_rows = []
header = ['tod', 'TP_4k']

#create the header
for measure in list_of_measures:
    for factype in list_of_FTs:
        header.append(factype + '_' + measure)
list_of_rows.append(header)

net_summary_df = pd.DataFrame(columns = header)
net_summary_df['tod'] = ft_summary_dict.keys()    
net_summary_df['TP_4k'] = net_summary_df['tod'].map(sound_cast_net_dict)
net_summary_df = net_summary_df.set_index('tod')
for key, value in ft_summary_dict.iteritems():
    for measure in list_of_measures:
        for factype in list_of_FTs:
            net_summary_df[factype + '_' + measure][key] = value[measure][factype]
net_summary_df.to_excel(excel_writer = writer, sheet_name = 'Network Summary')

#*******write out screenlines
screenline_df = pd.DataFrame()
screenline_df['Screenline'] = screenline_dict.keys()
screenline_df['Volumes'] = screenline_dict.values()
screenline_df.to_excel(excel_writer = writer, sheet_name = 'Screenline Volumes')

uc_vmt_df = pd.DataFrame(columns = uc_list, index = uc_vmt_dict.keys())
for colnum in range(len(uc_list)):
    for index in uc_vmt_dict.keys():
        uc_vmt_df[uc_list[colnum]][index] = uc_vmt_dict[index][colnum]
uc_vmt_df = uc_vmt_df.sort_index()
uc_vmt_df.to_excel(excel_writer = writer, sheet_name = 'UC VMT')

writer.save()

tod = {'am': '7to8', 'pm': '16to17'}
am_df = corridor_results(tod=tod['am'], my_project=my_project)
pm_df = corridor_results(tod=tod['pm'], my_project=my_project)

# combine am and pm into single CSV and export
corridor_df = pd.concat(objs=[am_df, pm_df])


1 * vauteq * (60/hdw)
296
296
1 * vauteq * (60/hdw)
296
296
296
296
1 * vauteq * (60/hdw)
296
1 * vauteq * (60/hdw)
296
296
1 * vauteq * (60/hdw)
296
296
4 * vauteq * (60/hdw)
296
{'9to10': [0.0, 249139.078125, 0.0, 1332614.75, 345869.53125, 416371.0625, 0.0, 33333.605469, 0.0, 334074.21875, 68178.414062, 70684.726562, 0.0, 13054.787109, 0.0, 180742.859375, 53388.496094, 62218.113281, 591214.0, 240675.484375, 133346.640625, 10880.87793], '16to17': [0.0, 355196.875, 0.0, 2008806.125, 656089.25, 757969.5625, 0.0, 52706.453125, 0.0, 431771.25, 118854.242188, 174585.359375, 0.0, 23489.882812, 0.0, 195081.640625, 65938.546875, 108117.5, 408866.15625, 116942.976562, 60281.140625, 0.0], '15to16': [0.0, 325486.125, 0.0, 2354568.25, 608361.5625, 679204.0, 0.0, 53198.875, 0.0, 598860.0625, 143454.25, 167352.34375, 0.0, 30206.669922, 0.0, 286205.28125, 90431.796875, 118026.664062, 351291.875, 100578.0625, 51833.269531, 0.0], '20to5': [0.0, 333673.5625, 0.0, 4675648.0, 1547449.5, 1871833.125, 0.0,

c:\Users\Brice\AppData\Local\Continuum\Anaconda\lib\site-packages\IPython\kernel\__main__.py:424: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\Users\Brice\AppData\Local\Continuum\Anaconda\lib\site-packages\IPython\kernel\__main__.py:425: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


c:\Users\Brice\AppData\Local\Continuum\Anaconda\lib\site-packages\IPython\kernel\__main__.py:428: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [84]:
# Add a column to identify the dataset
net_summary_df.insert(0,'summary', 'vmt_vht_vhd')
screenline_df.insert(0,'summary', 'screenlines')
corridor_df.insert(0, 'summary', 'travel_times')

ValueError: cannot insert summary, already exists

In [85]:
screenline_df['summary']

0     screenlines
1     screenlines
2     screenlines
3     screenlines
4     screenlines
5     screenlines
6     screenlines
7     screenlines
8     screenlines
9     screenlines
10    screenlines
11    screenlines
12    screenlines
13    screenlines
14    screenlines
15    screenlines
16    screenlines
17    screenlines
18    screenlines
19    screenlines
20    screenlines
21    screenlines
22    screenlines
23    screenlines
24    screenlines
25    screenlines
26    screenlines
27    screenlines
28    screenlines
29    screenlines
         ...     
36    screenlines
37    screenlines
38    screenlines
39    screenlines
40    screenlines
41    screenlines
42    screenlines
43    screenlines
44    screenlines
45    screenlines
46    screenlines
47    screenlines
48    screenlines
49    screenlines
50    screenlines
51    screenlines
52    screenlines
53    screenlines
54    screenlines
55    screenlines
56    screenlines
57    screenlines
58    screenlines
59    screenlines
60    scre

In [86]:
screenline_df['summary'] = 'screenlines'

In [90]:
df = screenline_df.append(net_summary_df)

In [5]:
from IPython.display import display, display_pretty, Javascript, HTML
from pandas_highcharts.core import serialize
from pandas_highcharts.display import display_charts
import matplotlib.pyplot as plt

## Transit Volume

In [78]:
tod = '8to9_board'
display_charts(transit_df[[tod]], title='8to9 AM Transit Boarding')

## Screenline Volumes

In [99]:
# Basic column plot
display_charts(screenline_df, title="AADT", kind='bar')

## VMT by Time of Day

In [140]:
# Basic column plot
display_charts(vmt.groupby('TP_4k').sum().ix[['am','md','pm','ev','ni']], 
               title="Daily VMT", kind='barh')

In [107]:
art_vmt = df[['arterial_vmt','TP_4k']].dropna()

In [108]:
art_vmt['arterial_vmt'] = art_vmt['arterial_vmt'].astype('int')

In [115]:
tod_arterial_delay = art_vmt.groupby('TP_4k').sum()

In [122]:
tod_arterial_delay.T[['am']]

TP_4k,am
arterial_vmt,5048713


In [129]:
vmt = df[['arterial_vmt','highway_vmt','TP_4k']].dropna()

In [131]:
vmt['highway_vmt'] = vmt['highway_vmt'].astype('int')
vmt['arterial_vmt'] = vmt['arterial_vmt'].astype('int')

In [138]:
vmt.groupby('TP_4k').sum()

,arterial_vmt,highway_vmt
TP_4k,,
am,5048713,6280168
md,12729522,13231420


In [20]:
    # Re-form screenlines for export to db
screenline_names = pd.read_json('inputs/screenline_dict.json',orient='values')
screenline_names['id'] = screenline_names.index
screenline_data = pd.DataFrame(screenline_dict.values(), index=screenline_dict.keys(),columns=['volume'])
screenline_data['id'] = screenline_data.index.astype('float64')
screenline_data = pd.merge(screenline_data, screenline_names)
screenline_data.fillna('',inplace=True)
screenline_data
screenline_data.index = screenline_data['Primary']+screenline_data['Secondary']
screenline_data.groupby(screenline_data.index).sum()[['volume']].T    # auburn has 2 values; need to be combined

NameError: name 'screenline_dict' is not defined

,volume
Agate Pass,21931.847168
Auburn,540059.208495
Bellevue/Redmond,353758.547852
Bothell,288796.929200
Cross-Sound,26878.771850
Gig Harbor,82340.605590
Kent,524684.136721
Kirkland/Redmond,416189.220702
Kitsap - North,78738.823122
Lynnwood/Bothell,266003.998536


In [ ]:
    # Re-form screenlines for export to db
screenline_names = pd.read_json('inputs/screenline_dict.json',orient='values')
screenline_names['id'] = screenline_names.index
screenline_data = pd.DataFrame(screenline_dict.values(), index=screenline_dict.keys(),columns=['volume'])
screenline_data['id'] = screenline_data.index.astype('float64')
screenline_data = pd.merge(screenline_data, screenline_names)
screenline_data.fillna('',inplace=True)
screenline_data.index = screenline_data['Primary']+screenline_data['Secondary']
# Need to combine the 2 Auburn screenlines, can't have duplicate column names in SQL


In [ ]:
#*******write out screenlines
screenline_df = pd.DataFrame()
screenline_df['Screenline'] = screenline_dict.keys()
screenline_df['Volumes'] = screenline_dict.values()
screenline_df.to_excel(excel_writer = writer, sheet_name = 'Screenline Volumes')

uc_vmt_df = pd.DataFrame(columns = uc_list, index = uc_vmt_dict.keys())
for colnum in range(len(uc_list)):
    for index in uc_vmt_dict.keys():
        uc_vmt_df[uc_list[colnum]][index] = uc_vmt_dict[index][colnum]
uc_vmt_df = uc_vmt_df.sort_index()
uc_vmt_df.to_excel(excel_writer = writer, sheet_name = 'UC VMT')

In [91]:
primary_screenlines = pd.read_json('inputs/screenline_dict.json',orient='values')

In [118]:
screenline_dict
df = pd.DataFrame(screenline_dict.values(), index=screenline_dict.keys(),columns=['volume'])

In [119]:
df['id'] = df.index.astype('float64')

In [120]:
primary_screenlines['id'] = primary_screenlines.index

In [152]:
screenline_data = [pd.merge(df, primary_screenlines].fillna('',inplace=True)

SyntaxError: invalid syntax (<ipython-input-152-28554ed3f419>, line 1)

In [148]:
# screenline_data.index=screenline_data['id']
screenline_data.fillna('',inplace=True)
screenline_data.index = screenline_data['Primary']+screenline_data['Secondary']
screenline_data[['volume']].T

TypeError: 'NoneType' object has no attribute '__getitem__'

In [135]:
screenline_data[['volume','Primary']].dropna()

,volume,Primary
id,,
22,227292.615723,Tukwila
35,544557.249999,Ship Canal
46,372797.083983,Mill Creek
23,68484.161012,Renton
44,288796.929200,Bothell
43,266003.998536,Lynnwood/Bothell
41,360121.720702,Seattle - North
4,304212.705080,Tacoma - East of CBD
29,480540.605469,Seattle - South of CBD


In [24]:
ft_summary_dict['7to8']

{'delay': {'arterial': 19899.488281,
  'connectors': -2e-06,
  'highway': 14518.820312},
 'vht': {'arterial': 75992.0,
  'connectors': 18303.074219,
  'highway': 56083.828125},
 'vmt': {'arterial': 2043806.0,
  'connectors': 280998.34375,
  'highway': 2332174.75}}

In [ ]:
ft_summary_dict['7to8']

In [72]:
dict_highway = {}
dict_arterial = {}
dict_connector = {}
for tod in tods:
    dict_highway[tod] = {}
    dict_arterial[tod] = {}
    dict_connector[tod] = {}
    for metric in ['delay', 'vht', 'vmt']:
        dict_highway[tod][metric] = ft_summary_dict[tod][metric]['highway']
        dict_arterial[tod][metric] = ft_summary_dict[tod][metric]['arterial']
        dict_connector[tod][metric] = ft_summary_dict[tod][metric]['connectors']

In [135]:
def dict_to_df(input_dict, metric, facilities=['highway', 'arterial']):
    '''converts triple-nested dict into Dataframe for a given facility type'''
    mydict = {}
    for tod in tods:
        mydict[tod] = {}
        for facility in facilities:
            mydict[tod][facility] = input_dict[tod][metric][facility]
    return pd.DataFrame(mydict)

In [136]:
ft_summary_dict['7to8']['delay']['arterial']

19899.488281

In [137]:
facility_results = {}
for metric in ['vmt','vht','delay']:
    facility_results[metric] = dict_to_df(input_dict=ft_summary_dict, metric=metric)

In [158]:
pd.DataFrame(facility_results['vmt'].T.sum(),columns=['vmt']).T

,arterial,highway
vmt,35886129.125,38338911.875


In [44]:
pd.DataFrame(dict_highway).sum()

10to14    8898275.509766
14to15    2521213.870117
15to16    2764830.496094
16to17    2689233.910157
17to18    2887156.410156
18to20    4046397.242188
20to5     5879430.305664
5to6      1083608.666870
6to7      1954758.977051
7to8      2402777.398437
8to9      2092054.938477
9to10     2141499.467773
dtype: float64

In [17]:
pd.DataFrame(net_stats['delay'].values(),index=net_stats['delay'].keys()).T

,arterial,connectors,highway
0,14617.649414,-0.000002,7630.592773


In [161]:
con

NameError: name 'con' is not defined

In [5]:
# Load the network summary script
from EmmeProject import *
from input_configuration import *

In [6]:
my_project = EmmeProject(r'D:\soundcast\soundcast\projects\LoadTripTables\LoadTripTables.emp')